In [1]:
#Import modules
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


2023-08-22 11:39:03.484172: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#Define train directory
train_dir = '/Users/david/Desktop/Pet facial expression Tensorflow/Pet Facial expression/Master Folder/train'
val_dir = '/Users/david/Desktop/Pet facial expression Tensorflow/Pet Facial expression/Master Folder/valid'
test_dir = '/Users/david/Desktop/Pet facial expression Tensorflow/Pet Facial expression/Master Folder/test'

In [3]:
# Instantiate ImageDataGenerator instances for training, validation and testing
train_datagen = ImageDataGenerator(rescale=1./255)
                                    #rotation_range=40,
                                    #width_shift_range=0.2,
                                    #height_shift_range=0.2,
                                    #shear_range=0.2,
                                    #zoom_range=0.2,
                                    #horizontal_flip=True,
                                    #fill_mode='nearest')
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(150, 150),
        batch_size=18,
        class_mode='categorical')

# Flow testing images in batches of 20 using test_datagen generator
testing_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(150, 150),
        batch_size=19,
        class_mode='categorical')

Found 1000 images belonging to 4 classes.
Found 36 images belonging to 4 classes.
Found 38 images belonging to 4 classes.


In [4]:
#Define model (first without transfer learning or fine-tuning; a baseline model)
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.MaxPooling2D(2,2),
      tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(512, activation='relu'),
      tf.keras.layers.Dense(4, activation='softmax')
    ])

    model.compile(optimizer = RMSprop(learning_rate = 0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])
  
    return model

In [5]:
# Create baseline model
baseline_model = create_model()

# Train the baseline model
baseline_history = baseline_model.fit(
      train_generator,
      steps_per_epoch=50,  
      epochs=10,
      validation_data=validation_generator,
      validation_steps=2)

Epoch 1/10


2023-08-22 11:39:21.363505: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - ETA: 0s - loss: 1.5070 - accuracy: 0.2500

2023-08-22 11:39:36.604473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 16s 298ms/step - loss: 1.5070 - accuracy: 0.2500 - val_loss: 1.3888 - val_accuracy: 0.1667
Epoch 2/10
50/50 [==============================] - 14s 287ms/step - loss: 1.3894 - accuracy: 0.2700 - val_loss: 1.3781 - val_accuracy: 0.3056
Epoch 3/10
50/50 [==============================] - 15s 292ms/step - loss: 1.3901 - accuracy: 0.2340 - val_loss: 1.3737 - val_accuracy: 0.2500
Epoch 4/10
50/50 [==============================] - 15s 303ms/step - loss: 1.3926 - accuracy: 0.2550 - val_loss: 1.3807 - val_accuracy: 0.3889
Epoch 5/10
50/50 [==============================] - 15s 307ms/step - loss: 1.3898 - accuracy: 0.2960 - val_loss: 1.3800 - val_accuracy: 0.3889
Epoch 6/10
50/50 [==============================] - 16s 312ms/step - loss: 1.3764 - accuracy: 0.3110 - val_loss: 1.3864 - val_accuracy: 0.2778
Epoch 7/10
50/50 [==============================] - 16s 317ms/step - loss: 1.3390 - accuracy: 0.3420 - val_loss: 1.3769 - val_accuracy: 0.2778
Epoch 8/10

In [7]:
#Evaluate baseline model
baseline_test_performance = baseline_model.evaluate(testing_generator)
print(f'Test set accuracy: {baseline_test_performance[1]*100}')

2023-08-22 11:55:42.944730: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2/2 [==============================] - 0s 90ms/step - loss: 1.6725 - accuracy: 0.2368
Test set accuracy: 23.68421107530594


In [8]:
#Define model (include transfer learning convolutional base)
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop

def create_model_transfer():
    conv_base = ResNet50V2(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

    model = tf.keras.models.Sequential()
    model.add(conv_base)
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(512, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.3))
    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(rate=0.2))
    model.add(tf.keras.layers.Dense(4, activation='softmax')) 

    conv_base.trainable = False

    model.compile(optimizer = RMSprop(learning_rate = 0.001),
                loss = 'categorical_crossentropy',
                metrics=['accuracy'])
  
    return model

In [9]:
#Create transfer learning model
transfer_model = create_model_transfer()

In [11]:
#Train transfer learning model
history = transfer_model.fit(
      train_generator,
      steps_per_epoch=50, # 1000 training images, batch size = 20, therefore 50 steps per epoch  
      epochs=10,
      validation_data=validation_generator,
      validation_steps=2 # 36 validation images, batch size = 18, therefore 2 validation setps per epoch
      )

Epoch 1/10


2023-08-22 11:56:02.060195: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - ETA: 0s - loss: 9.7893 - accuracy: 0.3820

2023-08-22 11:56:33.728687: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


50/50 [==============================] - 33s 622ms/step - loss: 9.7893 - accuracy: 0.3820 - val_loss: 2.3808 - val_accuracy: 0.3889
Epoch 2/10
50/50 [==============================] - 31s 619ms/step - loss: 2.8491 - accuracy: 0.5110 - val_loss: 2.7114 - val_accuracy: 0.3889
Epoch 3/10
50/50 [==============================] - 33s 665ms/step - loss: 1.8291 - accuracy: 0.5780 - val_loss: 2.3621 - val_accuracy: 0.4167
Epoch 4/10
50/50 [==============================] - 33s 651ms/step - loss: 1.3900 - accuracy: 0.6610 - val_loss: 2.1854 - val_accuracy: 0.3611
Epoch 5/10
50/50 [==============================] - 33s 656ms/step - loss: 0.9873 - accuracy: 0.7210 - val_loss: 2.8255 - val_accuracy: 0.4167
Epoch 6/10
50/50 [==============================] - 34s 678ms/step - loss: 0.8697 - accuracy: 0.7770 - val_loss: 2.4277 - val_accuracy: 0.4167
Epoch 7/10
50/50 [==============================] - 35s 696ms/step - loss: 0.5589 - accuracy: 0.8650 - val_loss: 3.0726 - val_accuracy: 0.5000
Epoch 8/10

In [12]:
#Evaluate transfer learning model
transfer_test_performance = transfer_model.evaluate(testing_generator)
print(f'Test set accuracy: {transfer_test_performance[1]*100}')

2023-08-22 12:01:46.326192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


2/2 [==============================] - 1s 525ms/step - loss: 5.7644 - accuracy: 0.4474
Test set accuracy: 44.736841320991516
